**Initialization**

The next few steps(or code snippets) initialize the colab environment for running the fast.ai course. Each line of code that added are in place to avoid any error, please do not change the position of any snippet if you do not know what they do.

Note:- You will get an error saying that the code is closing in the memory limit of 12Gb. You can click 'terminate other runtimes'.
       This error is popping up because the dataset is taking up most of the space, so don't worry
      

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmpx5p9pb_p/pubring.gpg' created
gpg: /tmp/tmpx5p9pb_p/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········


In [2]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [3]:
#@title Default title text
!wget http://files.fast.ai/data/dogscats.zip && unzip dogscats.zip -d data/

--2018-03-16 11:26:54--  http://files.fast.ai/data/dogscats.zip
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 857214334 (818M) [application/zip]
Saving to: ‘dogscats.zip.3’

dogscats.zip.3      100%[===================>] 817.50M  30.3MB/s    in 29s     

2018-03-16 11:27:22 (28.6 MB/s) - ‘dogscats.zip.3’ saved [857214334/857214334]

Archive:  dogscats.zip
replace data/dogscats/sample/train/cats/cat.2921.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [4]:
!pip install fastai

In [0]:
# https://opencv.org/
# pre installation
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2

In [0]:
# http://pytorch.org/
# pre installation
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [7]:
!pip install Pillow==4.0.0
!pip install PIL --allow-external PIL --allow-unverified PIL
!pip install image

  Using cached Pillow-4.0.0-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: Pillow 5.0.0
    Uninstalling Pillow-5.0.0:
      Successfully uninstalled Pillow-5.0.0
DEPRECATION: --allow-external has been deprecated and will be removed in the future. Due to changes in the repository protocol, it no longer has any effect.
DEPRECATION: --allow-unverified has been deprecated and will be removed in the future. Due to changes in the repository protocol, it no longer has any effect.
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL


## Image classification with Convolutional Neural Networks

In [0]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
#%reload_ext autoreload
#%autoreload 2
%matplotlib inline

In [0]:
# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [0]:
PATH = "data/dogscats/"
sz=224
arch=vgg16
bs=64

In [0]:
# Uncomment the below if you need to reset your precomputed activations
# !rm -rf {PATH}tmp

In [0]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))

In [13]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /content/.torch/models/vgg16_bn-6c64b313.pth
100%|██████████| 553507836/553507836 [00:15<00:00, 36409978.21it/s]


100%|██████████| 32/32 [00:27<00:00,  1.15it/s]


In [14]:
learn.fit(0.01, 3, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

[0.      0.0691  0.0324  0.98828]
[1.      0.05939 0.02524 0.99316]
 58%|█████▊    | 210/360 [00:05<00:04, 35.27it/s, loss=0.0635]

[2.      0.0455  0.02517 0.9917 ]



In [0]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

In [0]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=4)
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [17]:
learn.fit(1e-2, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

[0.      0.06097 0.03076 0.99121]
[1.      0.04505 0.02929 0.98975]



In [0]:
# learn.precompute=False

In [21]:
learn.fit(1e-2, 2, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

[0.      0.04555 0.03623 0.98877]
[1.      0.03997 0.02747 0.99023]



In [0]:
learn.unfreeze()

In [0]:
lr=np.array([1e-4,1e-3,1e-2])

In [26]:
learn.fit(lr, 1, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[0.      0.36205 0.26006 0.93897]



In [27]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

[0.      0.23013 0.16655 0.96631]
[1.      0.17236 0.11829 0.97315]
 61%|██████    | 220/360 [00:05<00:03, 40.32it/s, loss=0.156]

[2.      0.15418 0.10783 0.9751 ]
[3.      0.12572 0.08667 0.97949]
 98%|█████████▊| 351/360 [00:05<00:00, 58.67it/s, loss=0.129]

[4.      0.12569 0.07698 0.97998]
[5.      0.12158 0.07913 0.97754]
[6.      0.11388 0.07519 0.97949]



In [28]:
learn.fit(lr, 3, cycle_len=3)

HBox(children=(IntProgress(value=0, description='Epoch', max=9), HTML(value='')))

[0.      0.10282 0.06776 0.98193]
[1.      0.10399 0.06568 0.98096]
 59%|█████▉    | 212/360 [00:05<00:03, 37.06it/s, loss=0.0985]

[2.      0.10496 0.0651  0.98242]
[3.      0.09069 0.05853 0.98438]
 94%|█████████▍| 338/360 [00:06<00:00, 51.40it/s, loss=0.099]

[4.      0.09568 0.0601  0.98096]
[5.      0.08815 0.05687 0.9834 ]
[6.      0.09257 0.05688 0.9834 ]
  4%|▍         | 16/360 [00:00<00:15, 22.18it/s, loss=0.0895]

[7.      0.08885 0.05279 0.98535]
[8.      0.0849  0.05277 0.98486]



In [29]:
log_preds,y = learn.TTA()
accuracy(log_preds,y)

0.9855